# Get sequences around MAPT hairpin with mutations and snps in them and get MFE structures and ensemble structures 

folder written to in tmp: BuildEnsembleFromMutationsSNPs

This script will supply a fasta file with WT sequence around the hairpin and the start coordinates. Replace the WT base at the site of mutation or common SNP with the different base. 
Write all the sequences into a fasta file

In [61]:
tmpFolderToWrite="BuildEnsembleFromMutationsSNPs"

In [81]:
# This is the fasta file to read in for WT sequence
fastafile = "../data/MAPT_hairpin_CoordsForLaederachPrimer.fa"

In [82]:
# Open up the fasta file
with open(fastafile) as f:
    lines=[line.strip() for line in f]

In [83]:
# The second line will be the WT sequence
WT_seq=lines[1]
print(WT_seq)

AACGTCCAGTCCAAGTGTGGCTCAAAGGATAATATCAAACACGTCCCGGGAGGCGGCAGTGTGAGTACCTTCACACGTCCCATGCGCCGTGCTGTGGCTTGAATTATTAGGAAGTGGTGTGAGTGCGTACACTTGCGAGACACTGCATAGAATAAATCCTTCTTGGGCTCTCAGGATCTGGCTGCGACCTCTGGGTGAATG


In [84]:
# Get length of WT seq
wtseqLen = len(WT_seq)

In [85]:
# The first line will contain the start coordinate, separated by ,: and - [looks something like this >MAPT, chr17:46010343-46010543, MAPT hairpin ]
start_Coord=int(lines[0].split(",")[1].split(":")[1].split("-")[0])
print(start_Coord)

46010343


In [145]:
import pandas as pd

### Get 1000 suboptimal structures for the WT/Mut sequence using ViennaRNA 

In [137]:
mutationToWrite="rs116733906"

In [138]:
# Read in 1000 suboptimal structures 
with open("../tmp/"+tmpFolderToWrite+"/SuboptimalStructures_1000_MAPT_hairpin_LaederachPrimer_"+mutationToWrite+".txt") as f:
    wholefile = f.read().strip()   

In [139]:
# Split by sequences based on new line
splitByNewLine = wholefile.split("\n")
print(len(splitByNewLine))

1002


In [140]:
# Only grab lines that have . or ( or ) as the start character
splitByNewLine_OnlyDBs = [i for i in splitByNewLine if i[0] in [".","(",")"]]
print(len(splitByNewLine_OnlyDBs))

1000


In [141]:
# Only grab the first number of characters in the WT_seq 
onlyDBs = [i[0:wtseqLen] for i in splitByNewLine_OnlyDBs]
print(len(onlyDBs))

1000


In [142]:
# Only grab unique DBs
DBs_unique = list(set(onlyDBs))
print(len(DBs_unique))

997


In [143]:
# Write the suboptimal DB structures into a file
with open("../tmp/"+tmpFolderToWrite+"/SuboptimalStructures_1000_MAPT_hairpin_LaederachPrimer_"+mutationToWrite+"_UniqueDBs.db","w") as fw:
    fw.write("\n".join(DBs_unique))
    fw.write("\n")

### Get sequences around MAPT hairpin with muts/snps

In [3]:
# This is the write file to write mutations and snps into WT sequence
writefile = "MAPThairpin_LaederachPrimerSet_WTSeqsWithMutsAndSNPs.fa"

In [168]:
# Read in the mutation and common SNPs around MAPT hairpin file
muts_snps = pd.read_csv("../processed_data/MutsAndSNPsAroundTauHairpin_ForLaederachPrimerSet.bed",sep="\t",header=None)
muts_snps.head()

,0,1,2,3,4,5,6
0,chr17,46010357,46010358,CM1510268,T,C,DM
1,chr17,46010372,46010373,CM014584,A,C,DM
2,chr17,46010372,46010373,CM1411311,A,G,DM
3,chr17,46010374,46010375,CS003183,T,C,DM
4,chr17,46010378,46010379,CS140469,A,G,DM


In [ ]:
# Open up file to write 
with open("../tmp/"+tmpFolderToWrite+"/"+writefile,'w') as fw:
    # Go through every mutation
    for i in range(muts_snps.shape[0]):
        # Write the coordinates of mutation in the first line
        fw.write("> " + muts_snps.iloc[i,3] + "," + str(muts_snps.iloc[i,0])+":"+str(muts_snps.iloc[i,1])+"-"+str(muts_snps.iloc[i,2]))
        fw.write("\n")
        # Get the start coordinate, minus 1 from start Coord because its a 1-index and we want the 0-index
        start=muts_snps.iloc[i,1]-(start_Coord-1)
        print(start)
        # Obtain the sequence which includes the mutation/commonSNP 
        fw.write(WT_seq[0:start]+muts_snps.iloc[i,5]+WT_seq[start+1:])
        fw.write("\n")

### Get MFE structure for each sequence

In [ ]:
# Name file to read and write
readfile_struc = "MAPThairpin_LaederachPrimerSet_WTSeqsWithMutsAndSNPs.fa"
writefile_struc = "MAPThairpin_LaederachPrimerSet_MFEStructuresWithMutsAndSNPs.txt"
writefile_DB = "MAPThairpin_LaederachPrimerSet_MFEStructuresForMutsAndSNPS_unique.db"

In [ ]:
%%script bash -s "$tmpFolderToWrite" "$readfile_struc" "$writefile_struc"
# Run RNAfold to get MFE structures 
RNAfold -i ../tmp/${1}/${2} > ../tmp/${1}/${3}

In [ ]:
# Open and read the MFE structures file to extract the DB
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_struc) as f:
    lines = [line.strip() for line in f]

In [ ]:
# It looks like every 3rd position contains a dotbracket structure
db_strucs_withEnergy = [lines[i] for i in range(2,len(lines),3)]

In [ ]:
# Get length of WT seq
wtseqLen = len(WT_seq)

In [ ]:
# Since the sequences are all the same length, we can just grab the first n length 
db_strucs = [i[0:wtseqLen] for i in db_strucs_withEnergy]

In [ ]:
# only grab the unique structures
print(len(db_strucs))
db_strucs_uniq = list(set(db_strucs))
print(len(db_strucs_uniq))

In [ ]:
# Write the unique structures to a file
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_DB,'w') as fw:
    fw.write("\n".join(db_strucs_uniq))

### Get ensemble structures for each sequence

There are ~27 mutations/common SNPs around MAPT hairpin, if we generate 500 structures per mutation, then that is about 13500 structures which will probably decrease in size if we look for unique structures. 

I am going to do two sets using RNAsubopt one using the option deltaEnergy, and the other using stochBT option. 

In [86]:
# Name file to read and write
readfile_struc = "MAPThairpin_LaederachPrimerSet_WTSeqsWithMutsAndSNPs.fa"
writefile_Deltastruc = "MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy"
writefile_stochBT = "MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_stochBT"

#### Using RNAsubopt using deltaEnergy option

In [87]:
# Read in 2 delta energy file to check how many DBs are present per sequence 
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_Deltastruc+"-2.txt") as f:
    wholefile = f.read().strip()   

In [88]:
# Split by sequences based on new line
splitByNewLine = wholefile.split("\n")
print(len(splitByNewLine))

53284


In [89]:
# Only grab lines that have . or ( or ) as the start character
splitByNewLine_OnlyDBs = [i for i in splitByNewLine if i[0] in [".","(",")"]]
print(len(splitByNewLine_OnlyDBs))

53230


In [12]:
# Only grab the first number of characters in the WT_seq 
onlyDBs = [i[0:wtseqLen] for i in splitByNewLine_OnlyDBs]
print(len(onlyDBs))

53230


In [16]:
# Only grab unique DBs
DBs_unique = list(set(onlyDBs))
print(len(DBs_unique))

46560


In [21]:
# Write the suboptimal DB structures into a file
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_Deltastruc+"-2_UniqueDBs.db","w") as fw:
    fw.write("\n".join(DBs_unique))
    fw.write("\n")

In [90]:
# Split by sequences instead of new lines
splitBySeqs = wholefile.split(">")
print(len(splitBySeqs))

28


In [95]:
# For each seq, split by new line, then grab the first 500 lines
DBs_FromSeqs = []
for dbs in splitBySeqs[1:]:
    splitByNewLine = dbs.strip().split("\n")
    splitByNewLine_OnlyDBs = [i for i in splitByNewLine if i[0] in [".","(",")"]]
    # Only grab the first number of characters in the WT_seq for the first 500 DBs 
    onlyDBs = [i[0:wtseqLen] for i in splitByNewLine_OnlyDBs[0:500]]
    print(len(onlyDBs))
    DBs_FromSeqs.extend(onlyDBs)

395
346
500
328
233
500
500
500
500
500
500
371
500
500
500
500
300
306
500
306
500
436
500
500
500
266
500


In [96]:
print(len(DBs_FromSeqs))

11787


In [98]:
# Only grab unique DBs
DBs_unique = list(set(DBs_FromSeqs))
print(len(DBs_unique))

8955


In [99]:
# Write the suboptimal DB structures into a file
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_Deltastruc+"-2_Top500_UniqueDBs.db","w") as fw:
    fw.write("\n".join(DBs_unique))
    fw.write("\n")

#### Using RNAsubopt using stochBT option

In [53]:
numberDBs = "500"

In [54]:
# Read in stochBT file to check how many DBs are present per sequence 
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_stochBT+"-"+numberDBs+".txt") as f:
    wholefile = f.read().strip()   

In [55]:
# Split by sequences based on new line
splitByNewLine = wholefile.split("\n")
print(len(splitByNewLine))

13554


In [56]:
# Only grab lines that have . or ( or ) as the start character
splitByNewLine_OnlyDBs = [i for i in splitByNewLine if i[0] in [".","(",")"]]
print(len(splitByNewLine_OnlyDBs))

13500


In [57]:
# Only grab the first number of characters in the WT_seq 
onlyDBs = [i[0:wtseqLen] for i in splitByNewLine_OnlyDBs]
print(len(onlyDBs))

13500


In [58]:
# Only grab unique DBs
DBs_unique = list(set(onlyDBs))
print(len(DBs_unique))

13438


In [60]:
# Write the suboptimal DB structures into a file
with open("../tmp/"+tmpFolderToWrite+"/"+writefile_stochBT+"-"+numberDBs+"_UniqueDBs.db","w") as fw:
    fw.write("\n".join(DBs_unique))
    fw.write("\n")

In [28]:
3324/27

123

### Need to run ensembleRNA now on the ensembles created

In [ ]:
#ensemblerna -md MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db ../../data/MAPT_hairpin_CoordsForLaederachPrimer.fa ensembleRNA_MAPThairpin_LaederachPrimerSet_MapDBUsingSubOptimalDeltaEnergy-2
#ensemblerna -md MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db -d MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db ../../data/MAPT_hairpin_CoordsForLaederachPrimer.fa ensembleRNA_MAPThairpin_LaederachPrimerSet_DBAndMapDBUsingSubOptimalDeltaEnergy-2
#ensemblerna -r 50 150 -md MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db -d MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db ../../data/MAPT_hairpin_CoordsForLaederachPrimer.fa ensembleRNA_MAPThairpin_LaederachPrimerSet_Cluster50-150_DBAndMapDBUsingSubOptimalDeltaEnergy-2
#ensemblerna -r 50 150 -md MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_UniqueDBs.db ../../data/MAPT_hairpin_CoordsForLaederachPrimer.fa ensembleRNA_MAPThairpin_LaederachPrimerSet_Cluster50-150_MapDBUsingSubOptimalDeltaEnergy-2

### Write code to perform ensemble calculations for any mutation put in

In [169]:
import subprocess
import pandas as pd

def getPercentageChangesInEnsemble(WTseq,mutation,mutposition,projectedDBfile,rangeToCluster):
    # First get Mut Seq
    mutSeq = WTseq[0:mutposition]+mutation+WTseq[mutposition+1:]
    # Write MutSeq into fastafile
    with open("../tmp/mutationSeq.fa","w") as fw:
        fw.write("> Mutation_Position"+str(mutposition)+"_"+WTseq[mutposition]+"->"+mutation)
        fw.write("\n")
        fw.write(mutSeq)
        fw.write("\n")
        # Get 1000 suboptimal structures for the mutation
    with open("../tmp/SeqWithMutation_1000SuboptimalStructures.txt","w") as gw:
        subprocess.call(["RNAsubopt", "-p","1000", "-i","../tmp/mutationSeq.fa"], stdout=gw)
    # Read in 1000 suboptimal structures 
    with open("../tmp/SeqWithMutation_1000SuboptimalStructures.txt") as f:
        wholefile = f.read().strip()    
    # Split by sequences based on new line
    splitByNewLine = wholefile.split("\n")
    print(len(splitByNewLine))
    # Only grab lines that have . or ( or ) as the start character
    splitByNewLine_OnlyDBs = [i for i in splitByNewLine if i[0] in [".","(",")"]]
    print(len(splitByNewLine_OnlyDBs))
    # Only grab the first number of characters in the WT_seq 
    onlyDBs = [i[0:len(WTseq)] for i in splitByNewLine_OnlyDBs]
    print(len(onlyDBs))
    # Only grab unique DBs
    DBs_unique = list(set(onlyDBs))
    print(len(DBs_unique))
    # Write the suboptimal DB structures into a file
    with open("../tmp/SeqWithMutation_1000SuboptimalStructures_UniqueDBs.db","w") as fw:
        fw.write("\n".join(DBs_unique))
        fw.write("\n")
    # Call ensemblerna on this mutation db file
    subprocess.call(["ensemblerna", "-d","../tmp/SeqWithMutation_1000SuboptimalStructures_UniqueDBs.db","-md",projectedDBfile,"-r",str(rangeToCluster[0]),str(rangeToCluster[1]),"../tmp/mutationSeq.fa","../tmp/mutation_EnsembleRNA_output"])
    # Read in map db csv
    map_DB = pd.read_csv("../tmp/mutation_EnsembleRNA_output/mutationSeq_map.csv",sep=",",header=0)
    # Read in mut csv
    mut_DB = pd.read_csv("../tmp/mutation_EnsembleRNA_output/mutationSeq.csv",sep=",",header=0)
    # Return the numbers 
    Num_map_vector0 = map_DB[map_DB["vectorization"]=='[0]']["frequency"].values[0]
    Num_map_vector1 = map_DB[map_DB["vectorization"]=='[1]']["frequency"].values[0]
    Num_mut_vector0 = mut_DB[mut_DB["vectorization"]=='[0]']["frequency"].values[0]
    Num_mut_vector1 = mut_DB[mut_DB["vectorization"]=='[1]']["frequency"].values[0]
    return(Num_map_vector0,Num_map_vector1,Num_mut_vector0,Num_mut_vector1)

In [ ]:
# Go through all the mutations and SNPs and get number of structures for vector 0 and vector 1
# Read in the mutation and common SNPs around MAPT hairpin file
start_Coord=46010343
WT_seq="AACGTCCAGTCCAAGTGTGGCTCAAAGGATAATATCAAACACGTCCCGGGAGGCGGCAGTGTGAGTACCTTCACACGTCCCATGCGCCGTGCTGTGGCTTGAATTATTAGGAAGTGGTGTGAGTGCGTACACTTGCGAGACACTGCATAGAATAAATCCTTCTTGGGCTCTCAGGATCTGGCTGCGACCTCTGGGTGAATG"
muts_snps = pd.read_csv("../processed_data/MutsAndSNPsAroundTauHairpin_ForLaederachPrimerSet.bed",sep="\t",header=None)
with open("../tmp/Structures_forMuts.csv","w") as sw:
    for i in range(muts_snps.shape[0]):
        # Write the coordinates of mutation in the first line
        positionMut=muts_snps.iloc[i,1]-(start_Coord-1)
        mut = muts_snps.iloc[i,5]
        name_Mut = muts_snps.iloc[i,3]
        nums = getPercentageChangesInEnsemble(WT_seq,mut,positionMut,"../tmp/BuildEnsembleFromMutationsSNPs/MAPThairpin_LaederachPrimerSet_SuboptimalStructuresWithMutsAndSNPs_deltaEnergy-2_Top500_UniqueDBs.db",[50,80])
        toWrite = [str(name_Mut),str(nums[0]),str(nums[1]),str(nums[2]),str(nums[3])]
        fw.write(",".join(toWrite))
        fw.write("\n")